1) EXTRACT - 
AWS Cloud access and 2022 database load

In [0]:
# As I work at the Private Eduacation area, I decided to develop my MVP to understand the tendency of Brazillian teaching modality. For this analysis development, I used different year databases from Censo (2019, 2020, 2021 and 2024) to understand the impact of Coronavirus pandemic at teaching modalities.  

## GitHub blocked access key share

# AWS Access
access_key = 'xxxxxxxxx'
secret_key = 'xxxxxxxxxx'
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)

# AWS Region ID.
aws_region = "sa-east-1"
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3." + aws_region + ".amazonaws.com")

# 2022 Database load
df_cursos_22 = spark.read.csv('s3a://mvppuc/MICRODADOS_CADASTRO_CURSOS_2022.CSV',sep = ";",inferSchema=True,header=True)
df_cursos_22.show()

+------------+---------+---------+-----+-----+-----+------------+------------+----------+-----------+------------------------+---------------------------+-------+------+--------------------+--------+--------------------+--------------+------------------+--------------------+-----------------------+-----------------------+----------------------+----------------------+-----------------+-----------+--------------------+------------------+--------+-----------+------------------+-------------------+---------------+----------+-------------------+--------------+-------------------+-----------------+------------------------+-------------------------+---------------------+---------------+---------------------+-------------------+------------------------+------+----------+-----------+-------------+--------------+--------------+-----------------+-----------+------------------------+-------------------------+----------+-------------------------+--------------------+------------------+-------------

In [0]:
# Columns selection 
from pyspark.sql.functions import col
df_cursos_22_select = df_cursos_22.select(col("NU_ANO_CENSO"), col("CO_IES"), col("TP_MODALIDADE_ENSINO"),col("NO_CURSO"), col("QT_ING"))

2021 database load


In [0]:
# 2021 database load
df_cursos_21 = spark.read.csv('s3a://mvppuc/MICRODADOS_CADASTRO_CURSOS_2021.CSV',sep = ";",inferSchema=True,header=True)

In [0]:
# Columns selection
df_cursos_21_select = df_cursos_21.select(col("NU_ANO_CENSO"), col("CO_IES"), col("TP_MODALIDADE_ENSINO"),col("NO_CURSO"), col("QT_ING"))

In [0]:
#2022 and 2021 tables stack
df_union = df_cursos_22_select.unionAll(df_cursos_21_select)

In [0]:
# 2020 database load
df_cursos_20 = spark.read.csv('s3a://mvppuc/MICRODADOS_CADASTRO_CURSOS_2020.CSV',sep = ";",inferSchema=True,header=True)

In [0]:
# Columns selection 
df_cursos_20_select = df_cursos_20.select(col("NU_ANO_CENSO"), col("CO_IES"), col("TP_MODALIDADE_ENSINO"), col("NO_CURSO"), col("QT_ING"))

In [0]:
# Tables stack
df_union2 = df_union.unionAll(df_cursos_20_select)

In [0]:
# 2019 database load
df_cursos_19 = spark.read.csv('s3a://mvppuc/MICRODADOS_CADASTRO_CURSOS_2019.CSV',sep = ";",inferSchema=True,header=True)

In [0]:
#Columns selection
df_cursos_19_select = df_cursos_19.select(col("NU_ANO_CENSO"), col("CO_IES"), col("TP_MODALIDADE_ENSINO"), col("NO_CURSO"), col("QT_ING"))

In [0]:
# Tables stack
base_empilhada = df_union2.unionAll(df_cursos_19_select)

2) TRANSFORM



IES Base Load

In [0]:
# Database load
df_ies = spark.read.csv('s3a://mvppuc/MICRODADOS_ED_SUP_IES_2022.CSV',sep = ";",inferSchema=True,header=True)


In [0]:
# Columns selection
df_ies_select = df_ies.select(col("CO_IES"), col("NO_IES"), col("NO_UF_IES"), col("TP_ORGANIZACAO_ACADEMICA"), col("TP_CATEGORIA_ADMINISTRATIVA"))

2) TRANSFORM


In [0]:
# Transform categorical column to have only capital letters
from pyspark.sql import functions as F
base_empilhada2 = base_empilhada.withColumn('NO_CURSO', F.upper(F.col('NO_CURSO')))

In [0]:
# Transform categorical column to have only capital letters
df_ies_select2 = df_ies_select.withColumn('NO_UF_IES', F.upper(F.col('NO_UF_IES')))

In [0]:
# One CO_IES column rename to not generate a bug at the Join
df_ies_select3= df_ies_select2.withColumnRenamed("CO_IES", "CO_IES_df_eis_select2")

In [0]:
# Tables join
df_joined = base_empilhada2.join(df_ies_select3, base_empilhada2["CO_IES"] == df_ies_select3["CO_IES_df_eis_select2"], 'inner')


In [0]:
# Columns selection
df_joined2 = df_joined.select(col("NU_ANO_CENSO"), col("CO_IES"), col("NO_IES"), col("NO_UF_IES"), col("TP_ORGANIZACAO_ACADEMICA"), col("TP_CATEGORIA_ADMINISTRATIVA"), col("TP_MODALIDADE_ENSINO"),col("NO_CURSO"), col("QT_ING") )

3) LOAD


In [0]:
# Temp View Creation
temp_table_name = "BASE_EMPILHADA"
df_joined2.createOrReplaceTempView(temp_table_name)

In [0]:
# Permanent View Creation
permanent_table_name = "BASE_EMPILHADA"
df_joined2.write.format("parquet").saveAsTable(permanent_table_name)

Data Quality Analysis


In [0]:
%sql
select * from TABELA_COMPLETA

NU_ANO_CENSO,CO_IES,NO_IES,NO_UF_IES,TP_ORGANIZACAO_ACADEMICA,TP_CATEGORIA_ADMINISTRATIVA,TP_MODALIDADE_ENSINO,NO_CURSO,QT_ING
2022,417,UNIVERSIDADE CIDADE DE SÃO PAULO,SÃO PAULO,1,4,2,ABI - EDUCAÇÃO FÍSICA,11
2022,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,MATO GROSSO,1,1,2,ADMINISTRAÇÃO PÚBLICA,0
2022,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,MATO GROSSO,1,1,2,CIÊNCIA E TECNOLOGIA,0
2022,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,MATO GROSSO,1,1,2,CIÊNCIAS NATURAIS E MATEMÁTICA,0
2022,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,MATO GROSSO,1,1,2,MATEMÁTICA,0
2022,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,MATO GROSSO,1,1,2,PEDAGOGIA,0
2022,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,MATO GROSSO,1,1,2,TECNOLOGIA EDUCACIONAL,0
2022,2,UNIVERSIDADE DE BRASÍLIA,DISTRITO FEDERAL,1,1,2,ARTES VISUAIS,0
2022,2,UNIVERSIDADE DE BRASÍLIA,DISTRITO FEDERAL,1,1,2,EDUCAÇÃO FÍSICA,0
2022,2,UNIVERSIDADE DE BRASÍLIA,DISTRITO FEDERAL,1,1,2,FÍSICA,0


In [0]:
# Null count for column using agg()
contagem_nulos_por_coluna = df_joined2.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in df_joined2.columns])

contagem_nulos_por_coluna.show()

+------------+------+------+---------+------------------------+---------------------------+--------------------+--------+------+
|NU_ANO_CENSO|CO_IES|NO_IES|NO_UF_IES|TP_ORGANIZACAO_ACADEMICA|TP_CATEGORIA_ADMINISTRATIVA|TP_MODALIDADE_ENSINO|NO_CURSO|QT_ING|
+------------+------+------+---------+------------------------+---------------------------+--------------------+--------+------+
|           0|     0|     0|        0|                       0|                          0|                   0|       0|  4522|
+------------+------+------+---------+------------------------+---------------------------+--------------------+--------+------+



In [0]:
# Null replacement for 0
from pyspark.sql.functions import when, col

df_joined3 = df_joined2.withColumn('QT_ING', when(col('QT_ING').isNull(), 0).otherwise(col('QT_ING')))

In [0]:
# Null count for column using agg() after adjustment 
contagem_nulos_por_coluna = df_joined3.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in df_joined3.columns])

contagem_nulos_por_coluna.show()

+------------+------+------+---------+------------------------+---------------------------+--------------------+--------+------+
|NU_ANO_CENSO|CO_IES|NO_IES|NO_UF_IES|TP_ORGANIZACAO_ACADEMICA|TP_CATEGORIA_ADMINISTRATIVA|TP_MODALIDADE_ENSINO|NO_CURSO|QT_ING|
+------------+------+------+---------+------------------------+---------------------------+--------------------+--------+------+
|           0|     0|     0|        0|                       0|                          0|                   0|       0|     0|
+------------+------+------+---------+------------------------+---------------------------+--------------------+--------+------+



LOAD PT 2


In [0]:
# As it was necessary to replace null with 0 and I had already loaded the table, I deleted the existing to load the new one.


In [0]:
# Temp View Creation
temp_table_name = "BASE_EMPILHADA_V2"
df_joined3.createOrReplaceTempView(temp_table_name)

In [0]:
# Permanent View Creation
permanent_table_name = "BASE_EMPILHADAV2"
df_joined3.write.format("parquet").saveAsTable(permanent_table_name)